In [1]:
import pandas as pd #импортируем библиотеку для работы с данными в виде таблиц - `pandas`, и задаем ей сокращенное название `pd`
import numpy as np #импортируем библиотеку для работы с числами и массивами - `numpy`, и задаем ей сокращенное название `np`
from tqdm.notebook import tqdm #импортируем модуль `tqdm` из библиотеки `tqdm`, который позволяет отслеживать прогресс выполнения длительных операций, и конкретно используем здесь версию `tqdm` для использования в блокнотах `notebook`.

import torch #импортируем библиотеку для работы с нейронными сетями `PyTorch`
import torch.nn as nn #импортируем модуль `nn` из библиотеки `PyTorch`, который содержит функции для определения нейронных сетей.
from torch.utils.data import Dataset, DataLoader #импортируем классы `Dataset` и `DataLoader` из библиотеки `PyTorch`, которые позволяют загружать данные в нейронную сеть и обрабатывать их в виде батчей.
import pytorch_lightning as pl #импортируем библиотеку `PyTorch Lightning`, которая предоставляет удобный интерфейс для обучения моделей на `PyTorch`.

In [58]:
ratings = pd.read_csv('ratings.csv') # данная строка загружает данные из файла 'ratings.csv' и сохраняет их в переменную `ratings` в виде таблицы (DataFrame) с помощью функции `read_csv()` из библиотеки Pandas. Файл должен находиться в том же каталоге, что и файл с данным кодом.
ratings.head() #выводит первые 5 строк таблицы `ratings` с помощью функции `head()` из библиотеки Pandas.
ratings.shape #выводит размерности таблицы `ratings` в виде кортежа (количество строк, количество столбцов) с помощью атрибута `shape` у переменной `ratings`.

(27753444, 4)

In [59]:
#Первая строка кода генерирует случайную выборку пользователей, которые сделали оценки. В методе numpy.random.choice первым
#параметром передается уникальный список userId из DataFrame ratings, а вторым параметром передается количество 
#пользователей, которые будут выбраны (3% от общего числа пользователей в данном случае), и replace=False означает, что 
#каждый выбранный пользователь будет уникальным
rand_userIds = np.random.choice(ratings['userId'].unique(), 
                                size=int(len(ratings['userId'].unique())*0.03), 
                                replace=False)
#фильтрация DataFrame ratings по выбранным пользователей. Метод loc позволяет отбирать строки по условию, используя
#индексацию по меткам. Метод isin получает DataFrame рейтингов с строками, где столбец userId имеет значение, принадлежащее 
#выборке пользователей.
ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]
#выводит на экран количество строк данных и количество выбранных пользователей с помощью метода format. Здесь {} заменяются 
#на len(ratings) и len(rand_userIds) соответственно.
print('There are {} rows of data from {} users'.format(len(ratings), len(rand_userIds)))

There are 829177 rows of data from 8496 users


In [60]:
ratings.shape 

(829177, 4)

### Используя столбец timestamp, мы реализуем нашу стратегию разделения обучения и тестирования, используя методологию "оставить по одному". Для каждого пользователя самый последний обзор используется в качестве тестового набора (т.е. один из них исключается), в то время как остальные будут использоваться в качестве обучающих данных.

In [61]:
#Приведенный ниже код разделит наш набор данных о рейтингах на обучающий и тестовый наборы, используя методологию "оставить 
#все как есть".
ratings.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)

1494         1.0
1495         2.0
2088        11.0
2089        15.0
2090         4.0
            ... 
27752566    12.0
27752567     1.0
27752568     2.0
27752569    29.0
27752570    31.0
Name: timestamp, Length: 829177, dtype: float64

In [62]:
# Создается новый столбец 'rank_latest', который содержит ранг пользователя (userId) по времени (timestamp). 
#Дополнительно используется метод 'rank', который учитывает порядок появления записей (method='first') и сортирует в 
#порядке убывания(ascending=False).
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1] #Создаются обучающие данные (train_ratings), содержащие все записи, 
#кроме тех, где rank_latest равно 1 (знак, что это последняя оценка пользователя).
test_ratings = ratings[ratings['rank_latest'] == 1] #Создаются тестовые данные (test_ratings), содержащие записи последней
#оценки пользователей.
print(ratings.shape)
ratings.head()

(829177, 5)


,userId,movieId,rating,timestamp,rank_latest
1494,17,32,3.0,867664799,1.0
1495,17,780,5.0,867664799,2.0
2088,30,52,3.5,1182961438,11.0
2089,30,903,5.0,1182961417,15.0
2090,30,910,3.5,1182961465,4.0


In [65]:
# drop columns that we no longer need
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

### Чтобы преобразовать этот набор данных в набор данных неявной обратной связи, мы просто бинаризуем оценки таким образом, чтобы они были равны "1" (т.е. положительный класс). Значение "1" означает, что пользователь взаимодействовал с элементом.

In [66]:
train_ratings.loc[:, 'rating'] = 1 #Этот код устанавливает значение 1 для всех элементов столбца "rating" в датафрейме 
#train_ratings. В данном случае ":" означает, что нужно выбрать все строки в датафрейме, а "loc" используется для индексации
#по меткам (в данном случае мы используем метки строк ":", а столбца "rating"). Таким образом, мы заменяем рейтинги всех 
#фильмов в датафрейме на значение 1 (например, если это датафрейм для задачи бинарной классификации, где 1 обозначает 
#положительный класс, а 0 - отрицательный класс).

train_ratings.shape

(820681, 3)

### После того, как мы бинаризовали наш набор данных, мы обнаружили, что все образцы находятся в классе positive. Для того, чтобы наши модели были обучены правильно, нам также нужны отрицательные образцы, которые помогут указать, какие фильмы неинтересны пользователю. Хотя мы предположили, что отрицательные образцы - это те, которые пользователь не смотрел, это предположение может оказаться неверным. Однако, на практике это довольно хорошо работает.Наши 4 отрицательных выборки для каждого образца данных созданы для того, чтобы соотношение отрицательных и положительных образцов в нашем наборе данных было 4:1. Этот произвольный выбор соотношения был сделан, потому что он работает хорошо.

In [67]:
#Данный код извлекает все уникальные идентификаторы фильмов из столбца "movieId" в таблице "ratings" и сохраняет их в 
#массив "all_movieIds".
all_movieIds = ratings['movieId'].unique()


users, items, labels = [], [], [] # Этот код создает три пустых списка. 

user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId'])) #Этот код создает множество (set) под названием
#"user_item_set", которое содержит кортежи (tuples) - пары значений из двух столбцов "userId" и "movieId" из DataFrame 
#"train_ratings". Функция "zip()" создает последовательность кортежей соответствующих элементов из каждого переданного 
#списка (в данном случае два списка - "userId" и "movieId"). Кортежи добавляются в множество, которое удаляет любые 
#дублирующиеся кортежи. Таким образом, на выходе получаем множество всех уникальных комбинаций "userId" и "movieId" из 
#"train_ratings".

In [68]:
# Этот код используется для создания тренировочных датасетов в задаче рекомендации фильмов. Он проходится по каждому 
#пользователю и их взаимодействию с фильмами из датасета, затем генерирует отрицательные примеры для каждого пользователя. 
num_negatives = 4 #количество выбранных отрицательных примеров для каждого положительного примера.

for (u, i) in tqdm(user_item_set): #цикл перебирает каждую пару пользователь-фильм в датасете, используя библиотеку 
    #tqdm для отслеживания прогресса перебора. 
    users.append(u) #добавляет пользователя в список пользователей.
    items.append(i) #добавляет взаимодействующий фильм в список фильмов. 
    labels.append(1) #добавляет метку "1", чтобы указать, что пользователь взаимодействовал с этим фильмом, и это 
    #положительный пример для обучения. 
    for _ in range(num_negatives):#генерирует отрицательные примеры для каждого пользователя. 
        negative_item = np.random.choice(all_movieIds) #случайно выбирает фильм из всех фильмов в датасете. 
        while (u, negative_item) in user_item_set:# проверяет, что пользователь не взаимодействовал с этим фильмом до этого.
            #Если да, то случайным образом выбирается другой фильм до тех пор, пока не будет найден фильм, с которым 
            #пользователь еще не взаимодействовал.
            negative_item = np.random.choice(all_movieIds)
        users.append(u)#добавляет пользователя в список пользователей для этого отрицательного примера. 
        items.append(negative_item)#добавляет отрицательный фильм в список фильмов для этого примера. 
        labels.append(0) #добавляет метку "0" для этого отрицательного примера.

  0%|          | 0/820681 [00:00<?, ?it/s]

### Приведенный ниже класс просто инкапсулирует код, который мы написали выше, в класс набора данных PyTorch.

In [69]:
#Этот код представляет собой класс Dataset PyTorch для обучения модели рекомендации кинопроизведений MovieLens.
class MovieLensTrainDataset(Dataset):
    
    def __init__(self, ratings, all_movieIds): #определение конструктора класса с аргументами данных о рейтингах и списке id всех фильмов.
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds) #инициализация пользователей, фильмов и меток их рейтинга через вызов метода get_dataset.

    def __len__(self): #переопределение метода len()
        return len(self.users) #возвращает длину списка пользователей.
  
    def __getitem__(self, idx):# переопределение метода getitem().
        return self.users[idx], self.items[idx], self.labels[idx]#возвращает определенные пользователем, фильм и метку рейтинга для заданного индекса.

    def get_dataset(self, ratings, all_movieIds):#определение метода get_dataset() с аргументами оценками и списком id фильмов.
        users, items, labels = [], [], [] #инициализация пустых списков пользователей, фильмов и меток рейтинга.
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))# создание множества пар пользователя-фильма на основе данных оценок

        num_negatives = 4 #количество выбранных отрицательных примеров для каждого положительного примера.
        for u, i in user_item_set: #раскрытие множества пар на пользователей и фильмы.
            users.append(u) #добавление пользователя в список пользователей.
            items.append(i) #добавление фильма в список фильмов.
            labels.append(1) #добавление метки рейтинга в список меток рейтинга.
            for _ in range(num_negatives): # запускается цикл выбора случайных отрицательных примеров для каждого положительного примера
                negative_item = np.random.choice(all_movieIds)#выбор случайного фильма из списка всех фильмов.
                while (u, negative_item) in user_item_set:#проверка наличия уже оцененного или случайно выбранного пары пользователь-фильм.
                    negative_item = np.random.choice(all_movieIds)#выбор случайного фильма из списка всех фильмов.
                users.append(u)#добавление пользователя в список пользователей.
                items.append(negative_item)#добавление отрицательного примера в список фильмов.
                labels.append(0)#добавление метки рейтинга 0 для отрицательного примера.

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels) #возврат кортежа, состоящего из списков пользователей, фильмов и меток рейтинга, преобразованных в тензоры PyTorch.

### Мы создали модель, которая помогает нам понять, что пользователи любят смотреть в кино - боевики или романтические фильмы. Называется она Нейронная коллаборативная фильтрация или NCF. Мы расположили пользователей в двумерном пространстве, чтобы легче понимать их предпочтения. А для фильмов мы использовали отдельный слой встраивания элементов. В итоге мы можем идентифицировать, какие фильмы нравятся похожим пользователям, и на основе этого сделать рекомендации

In [70]:
class NCF(pl.LightningModule): #Создается класс `NCF`, который наследует функционал от `pl.LightningModule`.
    def __init__(self, num_users, num_items, ratings, all_movieIds): #Описывается метод `init`, который задает начальные параметры модели, такие как число пользователей (`num_users`), число элементов (`num_items`), массив оценок (`ratings`) и все идентификаторы фильмов (`all_movieIds`). 
        super().__init__()
        #Создаются слои `Embedding` для пользователей и элементов с разными размерностями параметров (`num_embeddings` и `embedding_dim`).
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        #Создаются слои `Linear` для прямого распространения данных (`in_features` и `out_features`).
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
    #Создается функция `forward`, которая описывает, как данные проходят через слои модели: сначала попадают на вход в `Embedding`, затем образуют единый вектор и проходят через все слои `Linear` с последующей применением функций активации. Наконец, для получения бинарной классификации применяется `nn.Sigmoid()`.
    def forward(self, user_input, item_input):
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)
        
        vector = torch.cat([user_embedded, item_embedded], dim=-1)
        
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))
        
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    #Создается метод `training_step`, который используется для вычисления ошибки при обучении. 
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss
    #Создается метод `configure_optimizers`, который возвращает оптимизатор - Adam.
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    #Создается метод `train_dataloader`, который используется для загрузки обучающих данных. Функция `DataLoader` загружает обучающие данные из `MovieLensTrainDataset` в пакетах (`batch_size=512`). `num_workers=0` означает, что данные загружаются синхронно (без использования нескольких процессов).
    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512, num_workers=0)

### Мы создаем экземпляр модели NCF, используя класс, который мы определили выше.

In [71]:
num_users = ratings['userId'].max()+1 #Эта строка кода создает переменную num_users, которая равна максимальному значению идентификатора пользователя в столбце "userId" из датафрейма ratings, увеличенному на единицу. Это нужно для того, чтобы определить количество пользователей, на которых будет обучаться модель.
num_items = ratings['movieId'].max()+1 #Эта строка кода создает переменную num_items, которая равна максимальному значению идентификатора фильма в столбце "movieId" из датафрейма ratings, увеличенному на единицу. Это нужно для того, чтобы определить количество фильмов, на которых будет обучаться модель.

all_movieIds = ratings['movieId'].unique() #Эта строка кода создает переменную all_movieIds, которая содержит уникальные идентификаторы фильмов из столбца "movieId" датафрейма ratings. Это нужно для того, чтобы передать эти значения в модель и использовать их при генерации рекомендаций.
model = NCF(num_users, num_items, train_ratings, all_movieIds) #Эта строка кода создает экземпляр класса NCF и передает в него параметры: num_users - количество пользователей, num_items - количество фильмов, train_ratings - обучающий датафрейм, all_movieIds - список уникальных идентификаторов фильмов. Эта модель будет использоваться для генерации рекомендаций на основе данных из train_ratings.

# Train

In [73]:
%%time #это функция в Jupyter Notebook, которая позволяет измерить время выполнения кода в ячейках.

trainer = pl.Trainer(max_epochs=5) #это создание объекта Trainer из PyTorch Lightning с параметром max_epochs, указывающим количество эпох, которые будут обучены.

trainer.fit(model) #это запуск тренировки модели с использованием созданного объекта Trainer и передача ему созданной ранее модели. Во время обучения модели Trainer будет обращаться к методам модели для получения и обновления ее параметров в каждую эпоху.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 2.3 M 
1 | item_embedding | Embedding | 1.5 M 
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.274    Total estimated model params size (MB)
C:\Users\Vlaso\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: total: 3h 12min 1s
Wall time: 1h 5min 7s


### Нам не нужно, чтобы пользователь кликал на каждом элементе списка рекомендаций. Чтобы сделать систему удобнее для пользователя, достаточно, чтобы он кликнул хотя бы на одном элементе из списка. Наше исследование показало, что если мы предлагаем пользователю 10 товаров, то, если выбранный им товар окажется в топ-10, мы сможем считать нашу систему успешной. Мы провели эксперимент, где выбрали случайные 99 товаров, с которыми пользователь не взаимодействовал, и добавили к ним один тестовый товар. Модель предсказала вероятности для каждого из этих 100 товаров, и мы выбрали 10 наиболее вероятных. Если тестовый товар был среди этих 10, мы считали это попаданием. Мы проверили все товары для всех пользователей, и получили среднее количество попаданий. Эта оценка называется коэффициент попадания при 10 и помогает оценить эффективность нашей системы.

In [74]:
#Создаем множество из кортежей (userId, movieId) для тестовых данных
test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

#Создаем словарь: для каждого пользователя описываем фильмы, с которыми он взаимодействовал
user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()

hits = [] #cоздаем пустой список для хранения результатов
for (u,i) in tqdm(test_user_item_set): # Итерируемся по всем юзерам и фильмам в тестовых данных, и для каждого из них строим рекомендации
    # Получаем список фильмов, с которыми взаимодействовал данный пользователь
    interacted_items = user_interacted_items[u]
    # Получаем множество фильмов, с которыми не взаимодействовал данный пользователь
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    # Случайным образом выбираем 99 фильмов, с которыми пользователь не взаимодействовал
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    # Формируем список фильмов, с которыми взаимодействовал пользователь и случайно выбранных фильмов
    # Добавляем в список исходный фильм i, для которого будем строить рекомендации
    test_items = selected_not_interacted + [i]
    # Строим предсказания рейтингов фильмов
    predicted_labels = np.squeeze(model(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
     # Отбираем топ-10 фильмов с наибольшими предсказанными рейтингами
    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    # Если исходный фильм i оказался в топ-10, добавляем 1 в список hits, т.е. рекомендация сработала
    if i in top10_items:
        hits.append(1)
    # Если нет, то добавляем 0
    else:
        hits.append(0)
# Выводим результат: среднее значение списка hits
print("The TOP @ 10 is {:.2f}".format(np.average(hits)))

  0%|          | 0/8496 [00:00<?, ?it/s]

The Hit Ratio @ 10 is 0.78


### это означает, что 78% пользователей были рекомендованы фактические товары (из списка из 10 товаров), с которыми они в конечном итоге взаимодействовали.